In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
holders_new = {}
holders_old = {}

In [21]:
# loads all of the distances' test data for both badge ages
# @return - df_all - a PANDAS DataFrame of all the test data
def load_all():
    
    # @param - dist - int indicating the distance between the receiver and transmitter during the test
    def load_dist(dist):
        raw = pd.read_csv('logs_oldvnew/' + str(dist) + 'ft_CSV/000.csv')
        return raw.loc[(raw['RSSI']>-70)]              # RSSI values below about -70 are too weak to be significant

    distances = [2, 3, 4, 5, 6, 8, 10]                # load all the distances' data in this test
    df_all = pd.concat([load_dist(d) for d in distances], keys=distances, names=["FT"])
    
    df_new = df_all.loc[df_all['MAC'].isin(holders_new)]   # filter by badge age / batch
    df_old = df_all.loc[df_all['MAC'].isin(holders_old)]
    return pd.concat([df_new, df_old], keys=['BATCH'])

In [22]:
data = load_all()